In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from tensorflow.keras.layers import (
    Conv2D,
    MaxPooling2D,
    Flatten,
    Dense,
    Dropout,
    Input,
    Lambda,
)
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.models import Sequential, Model
from sklearn.model_selection import train_test_split
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

c:\Users\ADMIN\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
data_dir = "./data/train/"
labels_path = "./data/labels.csv"
img_size = (224, 224, 3)

In [4]:
df = pd.read_csv(labels_path)
df.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [5]:
df["image_path"] = data_dir + df["id"] + ".jpg"
df = df.drop(columns="id")

In [6]:
breeds = sorted(df["breed"].unique().tolist())
len(breeds)

120

In [7]:
class_to_num = dict(zip(breeds, range(len(breeds))))

In [8]:
def image_processing(data: pd.DataFrame, image_size: tuple):
    X = np.zeros(
        [len(data), image_size[0], image_size[1], image_size[2]],
        dtype=np.uint8,
    )
    y = np.zeros([len(data), 1], dtype=np.uint8)
    for idx, row in tqdm(data.iterrows()):
        img_pixels = load_img(row["image_path"], target_size=image_size)
        X[idx] = img_pixels
        y[idx] = class_to_num[row["breed"]]
    print("Output Data Size: ", X.shape)
    print("Output Label Size: ", y.shape)
    return X, y

In [9]:
X, y = image_processing(df, img_size)

10222it [00:09, 1087.32it/s]

Output Data Size:  (10222, 224, 224, 3)
Output Label Size:  (10222, 1)


In [10]:
def extract_feature(image_size: tuple, data):
    input_layer = Input(image_size)
    processor = Lambda(preprocess_input)(input_layer)
    base_model = MobileNetV2(
        weights="imagenet", include_top=False, input_shape=image_size
    )(processor)
    maxPool = MaxPooling2D()(base_model)
    feature_extractor = Model(inputs=input_layer, outputs=maxPool)
    feature_maps = feature_extractor.predict(data, batch_size=64, verbose=1)
    print("Feature maps shape: ", feature_maps.shape)
    return feature_maps

In [11]:
features = extract_feature(img_size, X)

160/160 [==============================] - 82s 511ms/step
Feature maps shape:  (10222, 3, 3, 1280)


In [12]:
X_train, X_val, y_train, y_val = train_test_split(
    features, y, test_size=0.2, random_state=12
)

In [13]:
X_train = X_train / 255.0
X_val = X_val / 255.0

In [14]:
model = Sequential()
model.add(Input(X_train.shape[1:]))

model.add(Flatten())

model.add(Dense(128, activation="relu"))
model.add(Dropout(0.4))

model.add(Dense(len(breeds), activation="softmax"))
model.compile(

    loss="sparse_categorical_crossentropy",

    optimizer="adam",

    metrics=["accuracy"],
)


model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 11520)             0         
                                                                 
 dense (Dense)               (None, 128)               1474688   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 120)               15480     
                                                                 
Total params: 1,490,168
Trainable params: 1,490,168
Non-trainable params: 0
_________________________________________________________________


In [15]:
epochs = 40
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=epochs,
    batch_size=128,
)

Epoch 1/40
64/64 [==============================] - 1s 9ms/step - loss: 4.5293 - accuracy: 0.1290 - val_loss: 4.1073 - val_accuracy: 0.3032
Epoch 2/40
64/64 [==============================] - 1s 8ms/step - loss: 3.5724 - accuracy: 0.2978 - val_loss: 2.8964 - val_accuracy: 0.5134
Epoch 3/40
64/64 [==============================] - 0s 8ms/step - loss: 2.5566 - accuracy: 0.4436 - val_loss: 2.0016 - val_accuracy: 0.6367
Epoch 4/40
64/64 [==============================] - 0s 8ms/step - loss: 1.9414 - accuracy: 0.5380 - val_loss: 1.5350 - val_accuracy: 0.6944
Epoch 5/40
64/64 [==============================] - 0s 7ms/step - loss: 1.5983 - accuracy: 0.5921 - val_loss: 1.2841 - val_accuracy: 0.7183
Epoch 6/40
64/64 [==============================] - 0s 8ms/step - loss: 1.3612 - accuracy: 0.6402 - val_loss: 1.1268 - val_accuracy: 0.7359
Epoch 7/40
64/64 [==============================] - 1s 8ms/step - loss: 1.2269 - accuracy: 0.6650 - val_loss: 1.0405 - val_accuracy: 0.7374
Epoch 8/40
64/64 [==